In [1]:
import numpy as np
import random
import sys
import io
import os
import glob
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense, Input , Conv2D, ZeroPadding2D , Activation,Flatten
from sklearn.cross_validation import train_test_split
from keras.models import Model
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
%matplotlib inline


/home/nandwani_vaibhav/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nandwani_vaibhav/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
IPython.display.Audio("train/audio/yes/004ae714_nohash_0.wav")

In [2]:
X_SIZE = 16000
IMG_SIZE = 128
def spectrogram(filepath):
	framerate, wav_data = wavfile.read(filepath)

	window_length = 512
	window_shift = 121

	if len(wav_data) > X_SIZE:
		wav_data = wav_data[:X_SIZE]

	X = np.zeros(X_SIZE).astype('float32')
	X[:len(wav_data)] += wav_data
	spec = np.zeros((IMG_SIZE, IMG_SIZE)).astype('float32')

	for i in range(IMG_SIZE):
		start = i * window_shift
		end = start + window_length
		sig = np.abs(np.fft.rfft(X[start:end] * np.hanning(window_length)))
		spec[:,i] = (sig[1:IMG_SIZE + 1])[::-1]

	spec = (spec-spec.min())/(spec.max()-spec.min())
	spec = np.log10((spec * 100 + 0.01))
	spec = (spec-spec.min())/(spec.max()-spec.min()) - 0.5

	return spec

In [10]:
x =  spectrogram("train/audio/yes/004ae714_nohash_0.wav")
x.shape

(128, 128)

In [ ]:
x=[]
y=[]
classes={}
folder=glob.glob("train/audio/*")
for n,file in enumerate(folder):
    
     for fname in glob.glob(file+"/*"):
        print(fname)
        x.append(spectrogram(fname))
        y.append(n)

In [ ]:
x=np.array(x)
y=np.array(y)

#x=x.reshape((23682, 128, 128,1))

In [133]:
x=x.reshape((23682, 128, 128))

In [4]:
#len(x)
x.shape

(23682, 128, 128)

In [134]:
#np.save("data",x)
#np.save("labels",y)

In [3]:
x=np.load("data.npy")
y=np.load("labels.npy")

In [5]:
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.2,random_state=42)

In [6]:
ytrain = keras.utils.to_categorical(ytrain, 10)
ytest = keras.utils.to_categorical(ytest, 10)

In [58]:
def model(input_shape):
   

    X_input = Input(shape = input_shape)
    X = Conv1D(64, kernel_size=8, strides=2)(X_input)                                
    X = BatchNormalization()(X)                               
    X = Activation('relu')(X)                                 
    X = Dropout(0.6)(X)  
    print(X.shape)
    
    X = Conv1D(32, kernel_size=6, strides=2)(X)                                 
    X = BatchNormalization()(X)                                
    X = Activation('relu')(X)                                 
    X = Dropout(0.6)(X)  
    print(X.shape)
    
    X = Conv1D(16, kernel_size=6, strides=2)(X)                                 
    X = BatchNormalization()(X)                                 
    X = Activation('relu')(X)                                
    X = Dropout(0.6)(X)  
   
    print(X.shape) 
    
    X = GRU(units = 20, return_sequences = True)(X)   
    X = Dropout(0.8)(X)                                
    X = BatchNormalization()(X)                                  
    X = Dropout(0.6)(X)                                 
    
    print(X.shape)
    X=Flatten ()(X)
    X =Dense(10, activation = "softmax")(X)

    model = Model(inputs = X_input, outputs = X)
    
    return model

In [59]:


model=model(input_shape= (xtrain.shape[1:]))

(?, 61, 64)
(?, 28, 32)
(?, 12, 16)
(?, ?, 10)


In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 128, 128)          0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 61, 64)            65600     
_________________________________________________________________
batch_normalization_55 (Batc (None, 61, 64)            256       
_________________________________________________________________
activation_31 (Activation)   (None, 61, 64)            0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 61, 64)            0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 28, 32)            12320     
_________________________________________________________________
batch_normalization_56 (Batc (None, 28, 32)            128       
__________

In [61]:
opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [62]:
model.fit(xtrain, ytrain, batch_size = 5, epochs=2)

Epoch 1/2
18945/18945 [==============================] - 54s 3ms/step - loss: 0.3470 - acc: 0.8987
Epoch 2/2
18945/18945 [==============================] - 51s 3ms/step - loss: 0.3310 - acc: 0.9000


In [41]:
model.save("abc")

In [42]:
model.evaluate(xtest,ytest)

4737/4737 [==============================] - 3s 572us/step


[0.3253565415038523, 0.900000035750204]